In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import xgboost
import random
from datetime import datetime
from random import random
from pandas_profiling import ProfileReport
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KDTree
from datetime import datetime
from pulp import *
from decimal import Decimal
from scipy.optimize import minimize

In [2]:
df_history = pd.read_csv('./data/Demand_History.csv')
df = pd.read_csv('./data/exisiting_EV_infrastructure_2018.csv')


In [18]:
df_pred['2017'].sum()/ df_pred['2018'].sum()

0.8588497264171535

# Predict Demand for 2019 and 2020

In [37]:
df_pred = df_history
regr = LinearRegression()
#Lineare Regression
X =np.array([2010,2011,2012,2013,2014,2015,2016,2017,2018])
pred_2019 = []
pred_2020 = []
for row,cols in df_history.iterrows():
    y = []
    for num,(i) in enumerate(X):
        y.append(df_history[str(i)][row])  
    regr.fit(X.reshape(-1, 1), y)
    pred_2019.append(float(regr.predict(np.array([2019]).reshape(-1, 1))) )
    pred_2020.append(float(regr.predict(np.array([2020]).reshape(-1, 1))) )
df_pred['2019'] = pred_2019
df_pred['2020'] = pred_2020
df_pred.to_csv('./data/pred_linear_faktor.csv', index = False)
df_pred

,demand_point_index,x_coordinate,y_coordinate,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,0.5,0.5,0.352242,0.667932,0.958593,2.911901,4.338274,6.561995,8.454417,10.595324,13.119572,13.620020,15.278241
1,1,1.5,0.5,0.325940,0.591964,0.862652,2.589068,4.196034,5.745551,8.753195,11.126995,12.020091,13.244549,14.866537
2,2,2.5,0.5,0.373752,0.591890,0.969733,2.641432,3.541772,5.469161,8.414627,10.115336,14.018254,13.531706,15.212803
3,3,3.5,0.5,0.420686,0.584055,0.906547,2.378577,3.888121,5.846089,9.083868,12.424885,15.012302,15.092051,16.987237
4,4,4.5,0.5,0.475621,0.647940,0.981544,2.665400,4.218711,6.776609,8.851107,11.731131,16.355563,15.574265,17.517926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,4091,59.5,63.5,0.171015,0.334565,0.556055,1.373291,1.837586,2.517146,3.352280,4.149888,5.426193,5.457807,6.111190
4092,4092,60.5,63.5,0.041716,0.061741,0.131291,0.386540,0.755846,0.941116,1.107797,1.309479,2.057450,1.947587,2.186149
4093,4093,61.5,63.5,0.100895,0.180352,0.296299,0.705373,1.300220,1.608609,1.822806,2.333681,3.218519,3.192422,3.573868
4094,4094,62.5,63.5,0.155353,0.290825,0.557803,1.516066,2.399426,2.719197,4.494515,6.096858,6.262574,6.965030,7.813756


In [298]:
#Fill Zero Fields
years = ['2010','2011','2012','2013','2014','2015','2016','2017','2018']
for year in years:
    zeroCols = []
    for i,row in df_history.iterrows():
        if row[year] == 0:
            zeroCols.append(row['demand_point_index'])
    closest_Demands = []
    for i in zeroCols:
        closest_Demands.append([i,df_closest_demand.loc[i,'closest_supply_index'][1:]])
    closest_Demands

    for i in closest_Demands:
        df_history.loc[i[0],year] = np.sum([df_history.loc[j,year] for j in i[1]])/8
    df_history.reset_index(inplace=True)
    df_history.drop('level_0', axis = 1, inplace=True)


In [19]:
df_pred

,demand_point_index,x_coordinate,y_coordinate,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,0.5,0.5,0.352242,0.667932,0.958593,2.911901,4.338274,6.561995,8.454417,10.595324,13.119572,18.629276,23.583084
1,1,1.5,0.5,0.325940,0.591964,0.862652,2.589068,4.196034,5.745551,8.753195,11.126995,12.020091,17.654184,22.043718
2,2,2.5,0.5,0.373752,0.591890,0.969733,2.641432,3.541772,5.469161,8.414627,10.115336,14.018254,20.799537,27.665441
3,3,3.5,0.5,0.420686,0.584055,0.906547,2.378577,3.888121,5.846089,9.083868,12.424885,15.012302,23.046847,30.515047
4,4,4.5,0.5,0.475621,0.647940,0.981544,2.665400,4.218711,6.776609,8.851107,11.731131,16.355563,24.143251,32.190664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,4091,59.5,63.5,0.171015,0.334565,0.556055,1.373291,1.837586,2.517146,3.352280,4.149888,5.426193,7.742365,9.989230
4092,4092,60.5,63.5,0.041716,0.061741,0.131291,0.386540,0.755846,0.941116,1.107797,1.309479,2.057450,2.786423,3.610181
4093,4093,61.5,63.5,0.100895,0.180352,0.296299,0.705373,1.300220,1.608609,1.822806,2.333681,3.218519,4.388904,5.583748
4094,4094,62.5,63.5,0.155353,0.290825,0.557803,1.516066,2.399426,2.719197,4.494515,6.096858,6.262574,9.421834,11.878452


In [302]:
df_pred = pd.read_csv('./data/df_history.csv')

In [8]:
df_pred = df_history
polynomial_regression = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False),
    LinearRegression(),
)
X =np.array([2012,2013,2014,2015,2016,2017,2018])
pred_2019 = []
pred_2020 = []
for row,cols in df_history.iterrows():
    y = []
    for num,(i) in enumerate(X):
        y.append(df_history[str(i)][row])  
    polynomial_regression.fit(X.reshape(-1, 1), y)
    pred_2019.append(float(polynomial_regression.predict(np.array([2019]).reshape(-1, 1)))* 1.05)
    pred_2020.append(float(polynomial_regression.predict(np.array([2020]).reshape(-1, 1)))*1.05)
df_pred['2019'] = pred_2019
df_pred['2020'] = pred_2020
df_pred.to_csv('./data/pred_pol_factor.csv', index = False)

In [305]:
df_pred.to_csv('./data/df_pred_pol_filled.csv', index=False)

# Helper DataFrames, Run if variable files are missing

# Calculate Closest Supplypoint for each Demandpoint, add Demand Forecast and Difference compared to Previous year

In [9]:
#df_closest_supply = df_history['demand_point_index']
df_pred = pd.read_csv('./data/pred_pol_factor.csv')
X = np.array(df[['x_coordinate','y_coordinate']])
supply_id = []
supply_distance = []
tree = KDTree(X, leaf_size=2) 
for i in range(len(df_history)):           
    dist, ind = tree.query(np.array(df_history[['x_coordinate','y_coordinate']].iloc[i]).reshape(1, -1), k=100) 
    #print(f'Nearest Suppply: {ind}, Distance: {dist}')
    supply_id.append(ind[0])
    supply_distance.append(dist[0])
df_closest_supply = pd.DataFrame({'demand_point_index':df_history['demand_point_index'],'closest_supply_index':supply_id,'supply_distance':supply_distance,'Forecasted_Demand_2019':df_pred['2019'],'Rising_Demand_2019': df_pred['2019']-df_pred['2018'],'Forecasted_Demand_2020':df_pred['2020'],'Rising_Demand_2020': df_pred['2020']-df_pred['2019']})
#df_closest_supply['supply_distance'] = pd.DataFrame(supply_distance)
df_closest_supply

,demand_point_index,closest_supply_index,supply_distance,Forecasted_Demand_2019,Rising_Demand_2019,Forecasted_Demand_2020,Rising_Demand_2020
0,0,"[38, 88, 24, 57, 99, 60, 66, 34, 90, 74, 37, 6...","[8.004580252301615, 8.519681182197928, 14.0635...",16.300616,3.181044,19.047876,2.747259
1,1,"[38, 88, 24, 57, 99, 60, 66, 34, 74, 90, 37, 6...","[7.010519044978748, 8.628750195129735, 13.0756...",15.447411,3.427320,17.804541,2.357130
2,2,"[38, 88, 24, 57, 99, 60, 66, 74, 34, 90, 37, 6...","[6.018425836087249, 8.85018036051274, 12.08966...",18.199595,4.181341,22.345164,4.145569
3,3,"[38, 88, 24, 57, 99, 60, 66, 74, 90, 34, 37, 6...","[5.029465360150645, 9.175840827807518, 11.1062...",20.165991,5.153689,24.646769,4.480777
4,4,"[38, 88, 24, 57, 99, 60, 66, 74, 90, 34, 37, 6...","[4.045935500403834, 9.595124667293973, 10.1260...",21.125344,4.769781,26.000151,4.874807
...,...,...,...,...,...,...,...
4091,4091,"[16, 21, 2, 63, 47, 22, 81, 89, 72, 33, 43, 53...","[12.139734712501868, 13.368589677467302, 14.11...",6.774569,1.348376,8.068224,1.293655
4092,4092,"[16, 21, 2, 63, 47, 22, 81, 89, 72, 33, 43, 53...","[12.882291834070493, 14.083531845253647, 15.02...",2.438120,0.380670,2.915916,0.477796
4093,4093,"[16, 21, 2, 63, 47, 81, 89, 22, 72, 33, 43, 53...","[13.657735057702498, 14.831471555717654, 15.95...",3.840291,0.621772,4.509950,0.669659
4094,4094,"[16, 21, 2, 63, 47, 81, 89, 22, 33, 72, 43, 53...","[14.46077490712833, 15.607665673631342, 16.883...",8.244104,1.981531,9.594135,1.350030


In [269]:
#df_closest_supply = df_history['demand_point_index']
df_pred = pd.read_csv('./data/df_pred_pol.csv')
X = np.array(df_history[['x_coordinate','y_coordinate']])
supply_id = []
supply_distance = []
tree = KDTree(X, leaf_size=2) 
for i in range(len(df_history)):           
    dist, ind = tree.query(np.array(df_history[['x_coordinate','y_coordinate']].iloc[i]).reshape(1, -1), k=9) 
    #print(f'Nearest Suppply: {ind}, Distance: {dist}')
    
    supply_id.append(ind[0])
    supply_distance.append(dist[0])
df_closest_demand= pd.DataFrame({'demand_point_index':df_history['demand_point_index'],'closest_supply_index':supply_id,'supply_distance':supply_distance})
#df_closest_supply['supply_distance'] = pd.DataFrame(supply_distance)
df_closest_demand

,demand_point_index,closest_supply_index,supply_distance
0,0,"[0, 1, 64, 65, 2, 128, 129, 66, 130]","[0.0, 1.0, 1.0, 1.4142135623730951, 2.0, 2.0, ..."
1,1,"[1, 65, 2, 0, 64, 66, 129, 3, 128]","[0.0, 1.0, 1.0, 1.0, 1.4142135623730951, 1.414..."
2,2,"[2, 66, 1, 3, 67, 65, 4, 130, 0]","[0.0, 1.0, 1.0, 1.0, 1.4142135623730951, 1.414..."
3,3,"[3, 67, 4, 2, 68, 66, 131, 5, 1]","[0.0, 1.0, 1.0, 1.0, 1.4142135623730951, 1.414..."
4,4,"[4, 5, 3, 68, 69, 67, 2, 132, 6]","[0.0, 1.0, 1.0, 1.0, 1.4142135623730951, 1.414..."
...,...,...,...
4091,4091,"[4091, 4027, 4092, 4090, 4028, 4026, 3963, 409...","[0.0, 1.0, 1.0, 1.0, 1.4142135623730951, 1.414..."
4092,4092,"[4092, 4093, 4028, 4091, 4029, 4027, 4090, 396...","[0.0, 1.0, 1.0, 1.0, 1.4142135623730951, 1.414..."
4093,4093,"[4093, 4029, 4094, 4092, 4030, 4028, 4095, 409...","[0.0, 1.0, 1.0, 1.0, 1.4142135623730951, 1.414..."
4094,4094,"[4094, 4095, 4093, 4030, 4029, 4031, 4092, 396...","[0.0, 1.0, 1.0, 1.0, 1.4142135623730951, 1.414..."


# Demand to supply Distance DF

In [ ]:
supply = []
distance = []
demand = []
counter = 0
for i, row in df_closest_supply.iterrows():

    for j in range(100):
        supply.append(int(row.closest_supply_index[j]))
        distance.append(float(row.supply_distance[j]))
        demand.append(int(i))
        counter +=1
df_distance_demand_supply = pd.DataFrame({'Demand_Index': demand, 'Supply_Index':supply, 'Distance':distance})
supply = []
distance = []
demand = []
for i in range(4096):
    print(i)
    for j in range(100):
        demand.append(i)
        supply.append(j)
        distance.append(float(df_distance_demand_supply[(df_distance_demand_supply['Demand_Index'] == i) & (df_distance_demand_supply['Supply_Index'] == j)]['Distance']))
df_distance_demand_supply = pd.DataFrame({'Demand_Index': demand, 'Supply_Index':supply, 'Distance':distance})        
     

In [33]:
df_pred = pd.read_csv('./data/df_pred_pol_neu.csv')
df_pred

,demand_point_index,x_coordinate,y_coordinate,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,0.5,0.5,0.352242,0.667932,0.958593,2.911901,4.338274,6.561995,8.454417,10.595324,13.119572,18.629276,23.583084
1,1,1.5,0.5,0.325940,0.591964,0.862652,2.589068,4.196034,5.745551,8.753195,11.126995,12.020091,17.654184,22.043718
2,2,2.5,0.5,0.373752,0.591890,0.969733,2.641432,3.541772,5.469161,8.414627,10.115336,14.018254,20.799537,27.665441
3,3,3.5,0.5,0.420686,0.584055,0.906547,2.378577,3.888121,5.846089,9.083868,12.424885,15.012302,23.046847,30.515047
4,4,4.5,0.5,0.475621,0.647940,0.981544,2.665400,4.218711,6.776609,8.851107,11.731131,16.355563,24.143251,32.190664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,4091,59.5,63.5,0.171015,0.334565,0.556055,1.373291,1.837586,2.517146,3.352280,4.149888,5.426193,7.742365,9.989230
4092,4092,60.5,63.5,0.041716,0.061741,0.131291,0.386540,0.755846,0.941116,1.107797,1.309479,2.057450,2.786423,3.610181
4093,4093,61.5,63.5,0.100895,0.180352,0.296299,0.705373,1.300220,1.608609,1.822806,2.333681,3.218519,4.388904,5.583748
4094,4094,62.5,63.5,0.155353,0.290825,0.557803,1.516066,2.399426,2.719197,4.494515,6.096858,6.262574,9.421834,11.878452


In [32]:
df_pred1 = pd.read_csv('./data/pred_linear_faktor.csv')
df_pred1

,demand_point_index,x_coordinate,y_coordinate,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,0,0.5,0.5,0.352242,0.667932,0.958593,2.911901,4.338274,6.561995,8.454417,10.595324,13.119572,14.700858,20.039569
1,1,1.5,0.5,0.325940,0.591964,0.862652,2.589068,4.196034,5.745551,8.753195,11.126995,12.020091,14.342703,19.572900
2,2,2.5,0.5,0.373752,0.591890,0.969733,2.641432,3.541772,5.469161,8.414627,10.115336,14.018254,14.876648,20.379102
3,3,3.5,0.5,0.420686,0.584055,0.906547,2.378577,3.888121,5.846089,9.083868,12.424885,15.012302,16.735146,22.979559
4,4,4.5,0.5,0.475621,0.647940,0.981544,2.665400,4.218711,6.776609,8.851107,11.731131,16.355563,17.209426,23.603564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4091,4091,59.5,63.5,0.171015,0.334565,0.556055,1.373291,1.837586,2.517146,3.352280,4.149888,5.426193,5.841534,7.938911
4092,4092,60.5,63.5,0.041716,0.061741,0.131291,0.386540,0.755846,0.941116,1.107797,1.309479,2.057450,2.095118,2.855983
4093,4093,61.5,63.5,0.100895,0.180352,0.296299,0.705373,1.300220,1.608609,1.822806,2.333681,3.218519,3.404481,4.623057
4094,4094,62.5,63.5,0.155353,0.290825,0.557803,1.516066,2.399426,2.719197,4.494515,6.096858,6.262574,7.488203,10.201744


# Algorithm for Optimization Demand and Supply points

In [38]:
df_distance_demand_supply = pd.read_csv('./data/distance_demand_supply.csv')
df_pred = pd.read_csv('./data/pred_linear_faktor.csv')

In [39]:


model = LpProblem("Supply_Chain_Optimization", LpMinimize)
demand_supply_distance_dict = df_distance_demand_supply.to_dict('records')
df_2019 = df_pred['2019']
df_dict = df.to_dict('records')

x = LpVariable.dicts("Demand_Supply_", [(i['Demand_Index'],i['Supply_Index']) for i in demand_supply_distance_dict], lowBound=0, upBound=None, cat='continuous')
y = LpVariable.dicts('Supply_SC_',[(i['supply_point_index']) for i in df_dict ], lowBound=0, upBound=None, cat='Integer')

z = LpVariable.dicts('Supply_FC_',[(i['supply_point_index']) for i in df_dict ], lowBound=0, upBound=None, cat='Integer')



model += (lpSum([i['Distance'] * x[(i['Demand_Index'],i['Supply_Index'])]for i in demand_supply_distance_dict] )
         +400*lpSum([y[i['supply_point_index']]-i['existing_num_SCS'] for i in df_dict] + [z[i['supply_point_index']]-i['existing_num_FCS'] *1.5 for i in df_dict]))

#Constraint Number of build Chargers must be positive
for i in df_dict:
    model+= lpSum(y[i['supply_point_index']]) >= 0
    model+= lpSum(z[i['supply_point_index']]) >= 0

#Constraint Number of already buld chargers is minimum
for i in df_dict:
    model+= lpSum(y[i['supply_point_index']]) >= i['existing_num_SCS']
    model+= lpSum(z[i['supply_point_index']]) >= i['existing_num_FCS']

#Constraint Total Number of Chargers cant exceed parking spaces
for i in df_dict:
    model += lpSum(y[i['supply_point_index']]+z[i['supply_point_index']]) <= df_dict[i['supply_point_index']]['total_parking_slots']
#Constraint Charger must satify set demand for the year

model += lpSum([y[i['supply_point_index']]*200 for i in df_dict] + [z[i['supply_point_index']]* 400 for i in df_dict]) >= df_2019.sum()

model += lpSum([y[i['supply_point_index']]*200 for i in df_dict] + [z[i['supply_point_index']]* 400 for i in df_dict]) <= df_2019.sum()+400
  
#Constraint #1 Sum of Supply Satisfied per Demandpoint j, cant exceed Demandpoint j Capacity
for i in range(100):
    model += lpSum([x[(j,i)] for j in range(4096)]) <= y[i]*200 + z[i]* 400  
    
#Constraint #2 Sum of Demand  per partialdemandpoint i matrix equals predicted demand per whole demand point j
for i in range(4096):
    model += lpSum([x[(i, j)] for j in range(100)]) == df_2019[i]    

model.solve()



    

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sebastianbott/miniforge3/envs/kerasEnv/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/nl/g8256hnj7d31zb_rcxg2746w0000gn/T/f379753e14934c24bf6f986cb3761091-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nl/g8256hnj7d31zb_rcxg2746w0000gn/T/f379753e14934c24bf6f986cb3761091-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 4703 COLUMNS
At line 1235304 RHS
At line 1240003 BOUNDS
At line 1240204 ENDATA
Problem MODEL has 4698 rows, 409800 columns and 820400 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2.22955e+06 - 1.70 seconds
Cgl0004I processed model has 4267 rows, 406800 columns (200 integer (0 of which binary)) and 813800 elements
Cbc0012I Integer solution of 2233213.5 found by DiveCoefficient after 0 iterations and 0 nodes (

1

In [41]:
name = []
value = []
for v in model.variables():
    if 'Demand_Supply_' in v.name:
        name.append(str(v.name))
        value.append(float(v.varValue))
    
df_rework = pd.DataFrame({'Name':name, 'Satisfied_Demand': value})
df_rework['Demand'] = df_rework.Name.str.extract('(\d+)', expand=False).astype(int)
df_rework['Supply'] = df_rework.Name.str.extract('_(\d+)', expand=False).astype(int)
df_rework.sort_values(by=['Demand','Supply'], inplace = True)
df_rework

Demand = []
Supply=[]
for i in range(4096):
    for j in range(100):
        Demand.append(i)
        Supply.append(j)
Final_DF_2019 = pd.DataFrame({'Demand_Index':Demand,'Supply_Index':Supply,'Demand_Satisfied':df_rework['Satisfied_Demand']})
Final_DF_2019.to_csv('./Output/Highscore_tests/demand_to_supply_2019.csv', index=False)
name = []
value = []
for v in model.variables():
    if 'Supply_SC' in v.name:
        name.append(str(v))
        value.append(int(v.varValue))
df_SC_rework_2019 = pd.DataFrame({'Name':name, 'SC': value})
df_SC_rework_2019['Supply_Index']  = df_SC_rework_2019.Name.str.extract('(\d+)', expand=False).astype(int)
df_SC_rework_2019.sort_values(by=['Supply_Index'], inplace = True)
name = []
value = []
for v in model.variables():
    if 'Supply_FC' in v.name:
        name.append(str(v))
        value.append(int(v.varValue))
df_FC_rework_2019 = pd.DataFrame({'Name':name, 'FC': value})
df_FC_rework_2019['Supply_Index']  = df_SC_rework_2019.Name.str.extract('(\d+)', expand=False).astype(int)
df_FC_rework_2019.sort_values(by=['Supply_Index'], inplace = True)
supply_id = [i for i in range(100)]
supply_charger_2019 = pd.DataFrame({'supply_point_index':supply_id, 'existing_num_SCS':df_SC_rework_2019['SC'],'existing_num_FCS':df_FC_rework_2019['FC'] })
supply_charger_2019.reset_index(inplace=True)
supply_charger_2019.to_csv('./Output/Highscore_tests/supply_charger_2019.csv')


In [42]:

model2020 = LpProblem("Supply_Chain_Optimization", LpMinimize)
demand_supply_distance_dict = df_distance_demand_supply.to_dict('records')
df_2020 = df_pred['2020']
for i in range(100):
    supply_charger_2019.loc[supply_charger_2019['supply_point_index'] == i,'total_parking_slots'] = int(df.loc[df['supply_point_index'] == i,'total_parking_slots'])
df_dict = supply_charger_2019.reset_index()
df_dict['total_parking_slots'] = df['total_parking_slots']
df_dict = df_dict.to_dict('records')

x = LpVariable.dicts("Demand_Supply_", [(i['Demand_Index'],i['Supply_Index']) for i in demand_supply_distance_dict], lowBound=0, upBound=None, cat='continuous')
y = LpVariable.dicts('Supply_SC_',[(i['supply_point_index']) for i in df_dict ], lowBound=0, upBound=None, cat='Integer')

z = LpVariable.dicts('Supply_FC_',[(i['supply_point_index']) for i in df_dict ], lowBound=0, upBound=None, cat='Integer')



model2020 += (lpSum([i['Distance'] * x[(i['Demand_Index'],i['Supply_Index'])]for i in demand_supply_distance_dict] )
         +400*lpSum([y[i['supply_point_index']]-i['existing_num_SCS'] for i in df_dict] + [z[i['supply_point_index']]-i['existing_num_FCS'] *1.5 for i in df_dict]))

#Number of Built Chargers cant be less than zero
for i in df_dict:
    model2020+= lpSum(y[i['supply_point_index']]) >= 0
    model2020+= lpSum(z[i['supply_point_index']]) >= 0

#Constraint Number of already buld chargers is minimum
for i in df_dict:
    model2020+= lpSum(y[i['supply_point_index']]) >= i['existing_num_SCS']
    model2020+= lpSum(z[i['supply_point_index']]) >= i['existing_num_FCS']

#Constraint Total Number of Chargers cant exceed parking spaces
for i in df_dict:
    model2020 += lpSum(y[i['supply_point_index']]+z[i['supply_point_index']]) <= df_dict[i['supply_point_index']]['total_parking_slots']
#Constraint Charger must satify set demand for the year

model2020 += lpSum([y[i['supply_point_index']]*200 for i in df_dict] + [z[i['supply_point_index']]* 400 for i in df_dict]) >= df_2020.sum()
model2020 += lpSum([y[i['supply_point_index']]*200 for i in df_dict] + [z[i['supply_point_index']]* 400 for i in df_dict]) <= df_2020.sum()+400


  
#Constraint #1 Sum of Supply Satisfied per Demandpoint j, cant exceed Demandpoint j Capacity
for i in range(100):
    model2020 += lpSum([x[(j,i)] for j in range(4096)]) <= y[i]*200 + z[i]* 400   
    
#Constraint #2 Sum of Demand  per partialdemandpoint i matrix equals predicted demand per whole demand point j
for i in range(4096):
    model2020 += lpSum([x[(i, j)] for j in range(100)]) == df_2020[i]    

model2020.solve()

KeyboardInterrupt: 

In [ ]:

name = []
value = []
for v in model2020.variables():
    if 'Demand_Supply_' in v.name:
        name.append(str(v.name))
        value.append(float(v.varValue))
    
df_rework = pd.DataFrame({'Name':name, 'Satisfied_Demand': value})
df_rework['Demand'] = df_rework.Name.str.extract('(\d+)', expand=False).astype(int)
df_rework['Supply'] = df_rework.Name.str.extract('_(\d+)', expand=False).astype(int)
df_rework.sort_values(by=['Demand','Supply'], inplace = True)
df_rework

Demand = []
Supply=[]
for i in range(4096):
    for j in range(100):
        Demand.append(i)
        Supply.append(j)
Final_DF_2020 = pd.DataFrame({'Demand_Index':Demand,'Supply_Index':Supply,'Demand_Satisfied':df_rework['Satisfied_Demand']})
Final_DF_2020.to_csv('./Output/Highscore_tests/demand_to_supply_2020.csv', index=False)
name = []
value = []
for v in model2020.variables():
    if 'Supply_SC' in v.name:
        name.append(str(v))
        value.append(int(v.varValue))
df_SC_rework_2020 = pd.DataFrame({'Name':name, 'SC': value})
df_SC_rework_2020['Supply_Index']  = df_SC_rework_2020.Name.str.extract('(\d+)', expand=False).astype(int)
df_SC_rework_2020.sort_values(by=['Supply_Index'], inplace = True)
name = []
value = []
for v in model2020.variables():
    if 'Supply_FC' in v.name:
        name.append(str(v))
        value.append(int(v.varValue))
df_FC_rework_2020 = pd.DataFrame({'Name':name, 'FC': value})
df_FC_rework_2020['Supply_Index']  = df_FC_rework_2020.Name.str.extract('(\d+)', expand=False).astype(int)
df_FC_rework_2020.sort_values(by=['Supply_Index'], inplace = True)
supply_id = [i for i in range(100)]
supply_charger_2020 = pd.DataFrame({'supply_point_index':supply_id, 'existing_num_SCS':df_SC_rework_2020['SC'],'existing_num_FCS':df_FC_rework_2020['FC'] })
supply_charger_2020.reset_index(inplace = True)
supply_charger_2020.to_csv('./Output/Highscore_tests/supply_charger_2020.csv')



Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/sebastianbott/miniforge3/envs/kerasEnv/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/nl/g8256hnj7d31zb_rcxg2746w0000gn/T/9b8bddd2db9a47b3a5e4616422635c92-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/nl/g8256hnj7d31zb_rcxg2746w0000gn/T/9b8bddd2db9a47b3a5e4616422635c92-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 4703 COLUMNS
At line 1235304 RHS
At line 1240003 BOUNDS
At line 1240204 ENDATA
Problem MODEL has 4698 rows, 409800 columns and 820400 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2.19887e+06 - 1.11 seconds
Cgl0004I processed model has 4261 rows, 406788 columns (188 integer (0 of which binary)) and 813764 elements
Cbc0012I Integer solution of 2201212.8 found by DiveCoefficient after 0 iterations and 0 nodes (

# Optimize Demand

In [ ]:
model = LpProblem("Supply_Chain_Optimization", LpMinimize)
demand_supply_distance_dict = df_distance_demand_supply.to_dict('records')
df_2019 = df_pred['2019']

x = LpVariable.dicts("Demand_Supply_", [(i['Demand_Index'],i['Supply_Index']) for i in demand_supply_distance_dict], lowBound=0, upBound=None, cat='continuous')

model += lpSum([i['Distance'] * x[(i['Demand_Index'],i['Supply_Index'])]for i in demand_supply_distance_dict] )

#Constraint #1 Sum of Supply Satisfied per Demandpoint j, cant exceed Demandpoint j Capacity
for i in range(100):
    model += lpSum([x[(j,i)] for j in range(4096)]) <= temp_supply_2019[i]
    
#Constraint #2 Sum of Demand  per partialdemandpoint i matrix equals predicted demand per whole demand point j
for i in range(4096):
    model += lpSum([x[(i, j)] for j in range(100)]) == df_2019[i]
model.solve()

name = []
value = []
for v in model.variables():
    if 'Demand_Supply_' in v.name:
        name.append(str(v.name))
        value.append(float(v.varValue))
    
df_rework = pd.DataFrame({'Name':name, 'Satisfied_Demand': value})
df_rework['Demand'] = df_rework.Name.str.extract('(\d+)', expand=False).astype(int)
df_rework['Supply'] = df_rework.Name.str.extract('_(\d+)', expand=False).astype(int)
df_rework.sort_values(by=['Demand','Supply'], inplace = True)
df_rework

Demand = []
Supply=[]
for i in range(4096):
    for j in range(100):
        Demand.append(i)
        Supply.append(j)
Final_DF_2019 = pd.DataFrame({'Demand_Index':Demand,'Supply_Index':Supply,'Demand_Satisfied':df_rework['Satisfied_Demand']})
Final_DF_2019.to_csv('./Output/Highscore_tests/demand_to_supply_2019.csv', index=False)


model2020 = LpProblem("Supply_Chain_Optimization", LpMinimize)
demand_supply_distance_dict = df_distance_demand_supply.to_dict('records')
df_2020 = df_pred['2020']

x = LpVariable.dicts("Demand_Supply_", [(i['Demand_Index'],i['Supply_Index']) for i in demand_supply_distance_dict], lowBound=0, upBound=None, cat='continuous')

model2020 += lpSum([i['Distance'] * x[(i['Demand_Index'],i['Supply_Index'])]for i in demand_supply_distance_dict] )

#Constraint #1 Sum of Supply Satisfied per Demandpoint j, cant exceed Demandpoint j Capacity
for i in range(100):
    model2020 += lpSum([x[(j,i)] for j in range(4096)]) <= temp_supply_2020[i]
    
#Constraint #2 Sum of Demand  per partialdemandpoint i matrix equals predicted demand per whole demand point j
for i in range(4096):
    model2020 += lpSum([x[(i, j)] for j in range(100)]) == df_2020[i]
model2020.solve()

name = []
value = []
for v in model2020.variables():
    if 'Demand_Supply_' in v.name:
        name.append(str(v.name))
        value.append(float(v.varValue))
    
df_rework = pd.DataFrame({'Name':name, 'Satisfied_Demand': value})
df_rework['Demand'] = df_rework.Name.str.extract('(\d+)', expand=False).astype(int)
df_rework['Supply'] = df_rework.Name.str.extract('_(\d+)', expand=False).astype(int)
df_rework.sort_values(by=['Demand','Supply'], inplace = True)
df_rework

Demand = []
Supply=[]
for i in range(4096):
    for j in range(100):
        Demand.append(i)
        Supply.append(j)
Final_DF_2020 = pd.DataFrame({'Demand_Index':Demand,'Supply_Index':Supply,'Demand_Satisfied':df_rework['Satisfied_Demand']})
Final_DF_2020.to_csv('./Output/Highscore_tests/demand_to_supply_2020.csv', index=False)



In [ ]:
temp_df = pd.read_csv('./Output/Highscore_tests/supply_charger_2019.csv')
temp_supply_2019 = [int(row['Supply_SCS']*200 + row['Supply_FCS']*400) for i, row in temp_df.iterrows()]
df_supply_2019 = pd.DataFrame({'Supply_id':[i for i in range(100)], 'Capacity':temp_supply_2019 })
temp_df = pd.read_csv('./Output/Highscore_tests/supply_charger_2020.csv')
temp_supply_2020 = [int(row['Supply_SCS']*200 + row['Supply_FCS']*400) for i, row in temp_df.iterrows()]
df_supply_2020 = pd.DataFrame({'Supply_id':[i for i in range(100)], 'Capacity':temp_supply_2020 })
df_supply_2020